In [ ]:
print("Hello, this is a test of the notebook editing functionality.")

In [ ]:
DROP TABLE IF EXISTS amzl_mapping;
CREATE TEMP TABLE amzl_mapping AS (

SELECT
      location_id AS node
    , timezone
    , region
    , country
    , GETDATE() AS run_time_utc
    , CAST(CAST(GETDATE() AS DATE) AS TIMESTAMP) AS day_start_utc
    , CONVERT_TIMEZONE('UTC', timezone, GETDATE()) AS run_time_local
    , CONVERT_TIMEZONE('UTC', timezone, CAST(CAST(GETDATE() AS DATE) AS TIMESTAMP)) AS local_offset_utc
    
FROM
    "amzlanalytics"."perfectmile"."d_perfectmile_node_mapping_mdm"

WHERE
    1=1
    AND location_status = 'A'
    AND country IN('US', 'CA')
    AND business IN('AMZN', 'COLO')
    AND UPPER(region) IN('ROCKIES', 'UPSTATE NY') /*PILOT FILTER*/
    AND (location_type = 'DS' 
         OR
         location_id IN ('MCO5', 'ZYG1', 'ZYN9', 'XVV2', 'XVV3', 'XYT6', 'XLC1', 'XVC1', 'XNK2')
    )
    
);

In [ ]:
DROP TABLE IF EXISTS slam;
CREATE TEMP TABLE slam DISTKEY(shipment_id) AS (

-- assumption that slam leg only carries most recent slam data
WITH
leg AS (

SELECT
  LEG.leg_warehouse_id
, LEG.leg_destination_warehouse_id
, LEG.shipment_id
, LEG.package_id
, LEG.pickup_date
, LEG.estimated_arrival_date
, LEG.leg_ship_method
, LEG.request_timestamp
, ROW_NUMBER() OVER (PARTITION BY LEG.shipment_id, LEG.package_id
                     ORDER BY LEG.request_timestamp DESC, LEG.pickup_date DESC)
                     AS leg_rn
FROM
    "backlog_datasets"."atrops"."o_slam_packages_leg_live" LEG
INNER JOIN
    public.tv_shipments SHIP
ON 1=1
    AND LEG.request_timestamp > DATE_TRUNC('day', GETDATE())
    AND LEG.shipment_id = SHIP.shipment_id
QUALIFY
    leg_rn = 1
)

In [ ]:
DROP TABLE IF EXISTS gmp;
CREATE TEMP TABLE gmp AS (



-- WITH
-- gmp AS (

SELECT
    shipment_type,
    sender_id,
    tracking_id,
    ship_track_event_code,
    standard_carrier_alpha_code,
    supplement_code,
    tcda_container_id,
    parent_container_id,
    parent_container_type,
    status_node_id,
    load_id,
    status_date,
    status_date_timezone,
    edi_standard_name,
    status_code,
    reason_code,
    fulfillment_reference_id,
    marketplace_id,
    amazon_bar_code,
    fulfillment_shipment_id,
    estimated_arrival_date,
    promised_arrival_date,
    tss_ship_date,
    package_id,
    business_unit,
    ship_option,
    ship_method,
    actual_delivery_date,
    attempted_delivery_date,
    is_export_charge_prepaid,
    is_virtual_scan,
    access_point_id,
    seller_id,
    transport_shipment_id,
    amazon_reference_number,
    additional_reference_number,
    status_ref_target_type,
    predicted_delivery_date,
    estimated_delivery_date,
    origin_address_country_code
    manifest_id,
    service_type,
    dw_created_time,
    fulfillment_shipment_id || '#' || package_id AS shipment_package_pk,
    fulfillment_shipment_id || '#' || package_id || '#' || tracking_id as shipment_package_tracking_pk,
    ROW_NUMBER() OVER(PARTITION BY tracking_id ORDER BY status_date DESC) as status_date_rn,
    -- ROW_NUMBER() OVER(PARTITION BY tracking_id, status_date ORDER BY )
    1 as duplicate_rn,
    'GMP' as source
FROM
    "backlog_datasets"."amzlcore"."gmp_shipment_events_na" GMP
INNER JOIN
    public.tv_shipments SHIP
ON
    1=1
    AND GMP.fulfillment_shipment_id = SHIP.shipment_id
WHERE
    1=1
    AND GMP.dw_created_time > DATE_TRUNC('day', GETDATE())
);






DROP TABLE IF EXISTS gmp_vls;
CREATE TEMP TABLE gmp_vls sortkey(source) DISTKEY(shipment_package_pk) AS (
SELECT
      GMP.*
    , VLS.*
FROM
    gmp GMP
LEFT JOIN
    public.tv_load_summary VLS
ON
    1=1
    AND GMP.parent_container_id = VLS.vrid
);





DROP TABLE IF EXISTS stp_pk;
CREATE TEMP TABLE stp_pk DISTKEY(tracking_id) AS (

SELECT
DISTINCT
      tracking_id
    , shipment_package_pk
FROM
    gmp_vls
);

In [ ]:
DROP TABLE IF EXISTS public.tv_load_summary;
CREATE TABLE public.tv_load_summary DISTKEY(vrid) sortkey(final_destination) as (

SELECT
      vrid
    , origin
    , final_destination
    , origin_type
    , destination_type
    , lane
    , cpt
    , miles
    , origin_local_timezone
    , origin_scheduled_arrival
    , origin_calc_arrival
    , origin_arrival_late_group
    , origin_arrival_late_hrs
    , origin_arrival_reason
    , origin_scheduled_depart
    , origin_calc_depart
    , origin_departure_late_group
    , origin_depart_late_hrs
    , dest_local_timezone
    , dest_scheduled_arrival
    , dest_calc_arrival
    , dest_arrival_late_group
    , dest_arrival_late_hrs
    , dest_arrival_reason
    , account_id
    , report_day
    , vr_create_date
    -- , ROW_NUMBER() OVER(PARTITION BY vrid ORDER BY vr_create_date DESC) as duplicate_rn
    , 'VLS' as sub_source
FROM
    "andes"."ats-onestopshop"."v_load_summary_hourly"
WHERE
    1=1
    AND region_id = 'NA'
    AND dest_country IN ('US', 'CA')
    AND report_day > GETDATE() - INTERVAL '10 DAYS'
-- QUALIFY
--     ROW_NUMBER() OVER(PARTITION BY vrid ORDER BY vr_create_date DESC) = 1 -- this should not be neccesary

);


GRANT SELECT, INSERT, UPDATE, DELETE, TRUNCATE, REFERENCES, TRIGGER
ON TABLE public.tv_load_summary
TO PUBLIC;

In [ ]:

CREATE TEMP TABLE pse AS (
SELECT
      PSE."type"
    , PSE.package_id
    , PSE.package_id_type
    , PSE.forward_amazon_barcode
    , PSE.forward_tracking_id
    , PSE.forward_tcda_container_id
    , PSE.state_location_type
    , PSE.state_location_id
    , PSE.state_location_destination_id
    , PSE.state_location_source_id
    , PSE.state_status
    , PSE.state_time
    , PSE.triggerer_id
    , PSE.triggerer_id_type
    , PSE.dw_created_time
    , PSE.state_sub_status
    , PSE.comp_type
    , PSE.comp_reason
    , PSE.comp_state
    , PSE.reverse_amazon_barcode
    , PSE.reverse_tcda_container_id
    , PSE.reverse_tracking_id
    , PSE.execution_id
    , PSE.execution_id_type
    , PSE.state_location_destination_type
    , PSE.state_location_source_type
    , STP.shipment_package_pk
    , ROW_NUMBER() OVER(PARTITION BY PSE.forward_tracking_id ORDER BY PSE.state_time DESC) AS state_time_rn
    , ROW_NUMBER() OVER(PARTITION BY forward_tcda_container_id, forward_tracking_id ORDER BY state_time DESC, reverse_tracking_id) AS forward_tcda_rn
    , 'PSE' as source
    , 1 as duplicate_rn
FROM
    "backlog_datasets"."amzlcore"."package_systems_event_na" PSE
INNER JOIN
    public.stp_pk STP
ON
    STP.tracking_id = PSE.forward_tracking_id
                                                                                                                                                                                            -- INNER JOIN
                                                                                                                                                                                                -- (SELECT DISTINCT shipment_package_tracking_pk, tracking_id from gmp where tracking_id IS NOT NULL) GMP -- this join assumes that a tracking_id cannot be associated with >1 shipment_id
                                                                                                                                                                                            -- ON
                                                                                                                                                                                                -- PSE.forward_tracking_id = GMP.tracking_id

WHERE
    1=1
    AND dw_created_time > DATE_TRUNC('day', GETDATE()) - INTERVAL '6 DAYS'
)


In [ ]:


DROP TABLE IF EXISTS consolidated;
CREATE TEMP TABLE consolidated AS (



SELECT
  COALESCE(SGV.pk, PSE.shipment_package_pk) as pk
, COALESCE(SGV.source, PSE.source) as source
, COALESCE(SGV.event_timestamp, PSE.state_time) as event_timestamp
, COALESCE(SGV.duplicate_rn, PSE.duplicate_rn) as duplicate_rn
, SGV.shipment_id
, SGV.package_id
, SGV.warehouse_id
, SGV.route_delivery_group
, SGV.ship_method
, SGV.ship_option
, SGV.carrier_name
, SGV.zone
, SGV.pickup_date
, SGV.estimated_arrival_date
, SGV.request_timestamp
, SGV.external_estimated_arrival_date
, SGV.leg_warehouse_id
, SGV.leg_request_timestamp
, SGV.delivery_group_pickup_date
, SGV.shipment_type
, SGV.sender_id
, SGV.tracking_id
, SGV.ship_track_event_code
, SGV.standard_carrier_alpha_code
, SGV.supplement_code
, SGV.tcda_container_id
, SGV.parent_container_id
, SGV.parent_container_type
, SGV.status_node_id
, SGV.load_id
, SGV.status_date
, SGV.status_date_timezone
, SGV.edi_standard_name
, SGV.status_code
, SGV.reason_code
, SGV.amazon_bar_code
, SGV.fulfillment_shipment_id
, SGV.gmp_estimated_arrival_date
, SGV.gmp_package_id
, SGV.amazon_reference_number
, SGV.additional_reference_number
, SGV.status_ref_target_type
, SGV.service_type
, SGV.dw_created_time
, SGV.status_date_rn
, SGV.vrid
, SGV.origin
, SGV.final_destination
, SGV.origin_type
, SGV.destination_type
, SGV.vr_create_date

, PSE."type"
, PSE.package_id as pse_package_id
, PSE.package_id_type
, PSE.forward_amazon_barcode
, PSE.forward_tracking_id
, PSE.forward_tcda_container_id
, PSE.state_location_type
, PSE.state_location_id
, PSE.state_location_destination_id
, PSE.state_location_source_id
, PSE.state_status
, PSE.state_time
, PSE.triggerer_id
, PSE.triggerer_id_type
, PSE.dw_created_time AS pse_dw_created_time -- i am nulling this for now but need to come up with a better way to handle this
, PSE.state_sub_status
, PSE.comp_type
, PSE.comp_reason
, PSE.comp_state
, PSE.reverse_amazon_barcode
, PSE.reverse_tcda_container_id
, PSE.reverse_tracking_id
, PSE.execution_id
, PSE.execution_id_type
, PSE.state_location_destination_type
, PSE.state_location_source_type
-- , PSE.shipment_package_pk
, PSE.state_time_rn
, PSE.forward_tcda_rn
-- , PSE.source AS pse_source
-- , PSE.duplicate_rn


FROM
    slam_gmp_vls SGV
FULL OUTER JOIN
    pse PSE
ON
    SGV.source = PSE.source




);